In [ ]:
import calendar

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact

from resourcecode.weatherwindow import compute_weather_windows


# define a few constant that will be used for the plots
MONTH_NAMES = list(calendar.month_name)
HOURS_BY_YEARS = np.arange(1, 744 + 1)

hs_access_threshold = np.linspace(1, 3, 5)
hs_ref = np.array([1, 2, 3])
time_ref = np.array([24, 48, 72])

In [ ]:
# load the data
# in the future, data will be loaded using the client

hs = pd.read_csv(
    "../codes_to_migrate/matlab_codes/hs.csv",
    index_col=0,
    parse_dates=True,
).hs

In [ ]:
# compute weather windows for each month
weather_windows = {}
for month in range(1, 13):
    weather_windows[month] = compute_weather_windows(
        hs, month=month, hs_access_threshold=hs_access_threshold
    )

# Weibull Distribution Parameters

In [ ]:
# for each month, plot the weibull distribution result
@interact(month_name=MONTH_NAMES[1:])
def plot_weibull_distribution_parameters(month_name):
    month_index = MONTH_NAMES.index(month_name)
    weather_window_result = weather_windows[month_index]
    w = weather_window_result.weibull_distribution_result

    fig = make_subplots(rows=2, cols=1)
    fig.add_trace(
        go.Scatter(
            x=w.Ha,
            y=w.P,
            name="Fitted curve",
            text=f"x0 = {w.x0:0.5f}, b = {w.b:0.5f}, k = {w.k:0.5f}",
            showlegend=False,
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=w.Ha,
            y=w._MCFrHS,
            mode="markers",
            name="Experimental data",
            showlegend=False,
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(x=w._X, y=w._Y, mode="markers", showlegend=False), row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=w._X,
            y=np.log(np.log(1 / w.P)),
            showlegend=False,
            text=f"R = {w._residual:0.8f}",
        ),
        row=2,
        col=1,
    )

    fig.update_xaxes(title_text="Significant Wave Height (m)", row=1, col=1)
    fig.update_yaxes(title_text="Probability of Exceedance", row=1, col=1)

    fig.update_xaxes(
        title_text=r"$\ln\left(\ln\left(1 / P\right)\right)$", row=2, col=1
    )
    fig.update_yaxes(title_text=r"$\ln\left(H_s - x_0\right)$", row=2, col=1)
    fig.update_layout(
        title_text=(
            f"Weibull 3p fit − {month_name} (x0 = {w.x0:0.5f}, b = {w.b:0.5f}, k = {w.k:0.5f})"
        ),
        height=800,
    )
    return fig

# Mean Window Length

In [ ]:
@interact(month_name=MONTH_NAMES[1:])
def plot_mean_window_length(month_name):
    month_index = MONTH_NAMES.index(month_name)

    weather_window_result = weather_windows[month_index]
    weibull_dist_result = weather_window_result.weibull_distribution_result

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=weibull_dist_result.Ha,
            y=weather_window_result.tau,
            showlegend=False,
        ),
    )
    fig.update_xaxes(
        title_text="Significant Wave Height (m)",
        range=[0, 10],
    )
    fig.update_yaxes(
        title_text="Mean window Lenght (Hours)",
        range=[0, 240],
    )

    fig.update_layout(
        title_text=f"Mean window Length − {month_name}",
        height=800,
    )
    return fig

# Probability of Occurrence

In [ ]:
@interact(month_name=MONTH_NAMES[1:])
def plot_probability_of_occurrence(month_name):
    month_index = MONTH_NAMES.index(month_name)
    weather_window_result = weather_windows[month_index]

    fig = go.Figure()
    fig.add_traces(
        [
            go.Scatter(
                x=HOURS_BY_YEARS,
                y=weather_window_result.PT[i],
                name=f"{ha} m",
            )
            for i, ha in enumerate(hs_access_threshold)
        ]
    )

    fig.update_xaxes(
        title_text="Duration of Weather Window (Hours)",
    )
    fig.update_yaxes(
        title_text="Probability of Occurrence",
    )

    fig.update_layout(
        title_text=f"Probability of Occurrence − {month_name}",
        height=800,
    )
    return fig

# Monthly evolution of number of events

In [ ]:
@interact(time=time_ref, hs=hs_ref)
def plot_evolution_of_number_of_events(time, hs):
    fig = make_subplots(rows=3, cols=1)

    itime = np.where(HOURS_BY_YEARS == time)[0][0]
    iha = np.where(hs_access_threshold == hs)[0][0]

    nb_events = []
    nb_access_hours = []
    nb_waiting_hours = []
    for r in weather_windows.values():
        nb_events.append(r.number_events[iha, itime])
        nb_access_hours.append(r.number_access_hours[iha, itime])
        nb_waiting_hours.append(r.number_waiting_hours[iha, itime])

    fig.add_trace(
        go.Scatter(x=MONTH_NAMES[1:], y=nb_events, name="Number of events"),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=MONTH_NAMES[1:],
            y=nb_access_hours,
            name="Access time",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=MONTH_NAMES[1:],
            y=nb_waiting_hours,
            name="Waiting time",
        ),
        row=3,
        col=1,
    )

    fig.update_yaxes(title_text="Number of Events", row=1, col=1)
    fig.update_yaxes(title_text="Hours", row=2, col=1)
    fig.update_yaxes(title_text="Hours", row=3, col=1)
    fig.update_xaxes(title_text="Months", row=3, col=1)

    fig.update_layout(
        title_text=f"Weibull 3p fit − Hs = {hs}m, {time} hours window",
        height=1200,
    )

    return fig